# Guide: Building a Loan with _cred_

<a href="https://colab.research.google.com/github/jordanhitchcock/cred_guides/blob/master/loan_schedule_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

_cred_ is an open source Python package for modeling commerial real estate debt. This vignette is a step-by-step 
guide to creating a loan object and calculating the pro forma debt schedule including dates, payments, and coverage metrics. In this tutorial, we are going to build a debt model for a loan originated today based on live end of day interest rates. Click "Open in Colab" to run this notebook in your browser.

This guide was written for readers that are familiar with CRE debt structures but are new to both Python and the _cred_ package. It provides full support to follow along including environment setup and installation. For directions on how to set up the environment, start by skipping down the section called [Setting up the Python environment](#pythonenv) at the bottom.
__Links:__ [__Github__](https://github.com/jordanhitchcock/cred), [__Docs__](https://cred.readthedocs.io/en/latest/)

## Installation and imports
One of the great advantages of _cred_ over Excel is the ability to easily connect to outside sources of information such as cash flow projections pulled from leasing software or third-party market data providers. Here we use a helper package called creatively called `treasurydotgovyc` to pull the end-of-day Treasury yield curve from treasury.gov.

In [1]:
!pip install python-dateutil > /dev/null
!pip install git+https://github.com/jordanhitchcock/treasurydotgovyc > /dev/null
!pip install git+https://github.com/jordanhitchcock/cred > /dev/null
from datetime import date
from dateutil.relativedelta import relativedelta
from treasurydotgovyc import YieldCurve
from cred import FixedRateBorrowing, Monthly

  Running command git clone -q https://github.com/jordanhitchcock/treasurydotgovyc /private/var/folders/3x/0m97cfv94pq0bw4hq39_6jv80000gn/T/pip-req-build-33re0jh1
  Running command git clone -q https://github.com/jordanhitchcock/cred /private/var/folders/3x/0m97cfv94pq0bw4hq39_6jv80000gn/T/pip-req-build-zybzblyt


<a id='loanschedule'><a/>
## Creating the loan schedule

The three lines below are all it takes to fetch the most recent yield curve, initialize a loan, and print the cash flow schedule!

In [2]:
yc = YieldCurve()

loan = FixedRateBorrowing(start_date=date.today(),
                          end_date=date.today() + relativedelta(months=12),
                          freq=Monthly(1),
                          initial_principal=100_000_000,
                          coupon=yc.yield_for_delta(relativedelta(years=1)) + 0.02)

loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-04-16,2020-05-16,2020-05-16,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
1,2020-05-16,2020-06-16,2020-06-16,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
2,2020-06-16,2020-07-16,2020-07-16,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
3,2020-07-16,2020-08-16,2020-08-16,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
4,2020-08-16,2020-09-16,2020-09-16,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
5,2020-09-16,2020-10-16,2020-10-16,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
6,2020-10-16,2020-11-16,2020-11-16,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
7,2020-11-16,2020-12-16,2020-12-16,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
8,2020-12-16,2021-01-16,2021-01-16,100000000,0.0219,188583.333333,0,1.885833e+05,100000000


Let's break that down (show/hide cell output by selecting and the section above and typing "o").
1. We initialize a new `YieldCurve` object which we will use later on to calculate the coupon rate. When this object is initialized, it sends a get request to [treasury.gov](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/XmlView.aspx?data=yield) to get the most recent EoD yield curve and parses the XML response.

2. Next we create a new fixed rate loan and initialize it with the following values:

Parameter | Value | Details
:- | :- | :-
`start_date` | Today | `date.today()` returns today's date.
`end_date` | One year from today | This example loan has a one year term. Finds today's date and adds one year.
`freq` | 1 month | Date offset object that defines interest period frequency.
`initial_principal` | $100 million | Initial principal balance. Interest only, so the principal amount does not change.
`coupon` | Interp. T rate + 2.00% | `yc.yield_for_delta` asks the `yc` object to return the interpolated Treasury rate for the `relativedelta` offset parameter, which in this case is 1 year to match the term of the loan.

3. Finally, the `.schedule()` method returns a `pd.DataFrame` with the loan schedule.


## Creating a more complicated loan
The example above is incredibly simple. In practice, loan schedules often aren't as tidy. The _cred_ package simplifies those complexities. This next example highlights how to handle common issues either through convenience functions or custom implementation.

First, let's redefine our starting assumptions and recreate the first loan object. We will modify this object to look more like a typical lifeco loan that you might actually see in the wild.

In [3]:
closing_date = date.today()
term = Monthly(months=12)
maturity = closing_date + term
freq = relativedelta(months=1)
initial_principal = 100_000_000
coupon = yc.yield_for_delta(term) + 0.02

loan = FixedRateBorrowing(start_date=closing_date,
                         end_date=maturity,
                         freq=freq,
                         initial_principal=initial_principal,
                         coupon=coupon)

The term of our initial example conveniently equals exactly one year. Let's change the loan to a two year term beginning on the 15th of next month, rolling on the last day of the month, and maturing on the 15th day of the month in which it matures. This means we will have beginning and ending stub interest periods.

In [4]:
loan.start_date = date.today() + relativedelta(months=1, day=15) 
loan.first_reg_start = closing_date + relativedelta(months=1, day=31) # If the next month has < 31 days, will set to last day
loan.end_date = loan.start_date + Monthly(months=24)

loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,100000000,0.0219,97333.333333,0,9.733333e+04,100000000
1,2020-05-31,2020-06-30,2020-06-30,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
2,2020-06-30,2020-07-31,2020-07-31,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
3,2020-07-31,2020-08-31,2020-08-31,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
4,2020-08-31,2020-09-30,2020-09-30,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
5,2020-09-30,2020-10-31,2020-10-31,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
6,2020-10-31,2020-11-30,2020-11-30,100000000,0.0219,182500.000000,0,1.825000e+05,100000000
7,2020-11-30,2020-12-31,2020-12-31,100000000,0.0219,188583.333333,0,1.885833e+05,100000000
8,2020-12-31,2021-01-31,2021-01-31,100000000,0.0219,188583.333333,0,1.885833e+05,100000000


<div class="alert alert-info">

**Note on initialization and stored values:** We are directly modifying properties on Borrowing object, but for each property that we are change there is parameter (usually) with the same name that you can define at initialization.
    
Period values are not stored, so you can safely change any Borrowing property and the schedule will rebuild as expected. For performance, period values are cached during execution of the _.schedule()_ method. You can manually tell it to cache values by using the _with_ context manager, but be careful about changing object properties.

</div>

Great, now we have our dates all line up with the expected closing timeline. Notice that by default the payment date for the initial stub period is on the closing date which is common in practice. However, payment dates are unadjusted and might currently fall on weekends or holidays. Let's fix this by defining a modified following convention with New York business days.

In [5]:
from cred import modified_following, FederalReserveHolidays

loan.adjust_pmt_date = modified_following
loan.holidays = FederalReserveHolidays()

Note that we adjusted the payment dates but not the interest period calculation dates. Depending on the specific definitions in the loan documents, the interest period starting and ending dates may or may not also follow an adjustment convention. This largely depends on the lending market, lender, and counsel drafting the loan docs.

The default interest calculation uses an Actual / 360 day count convention. Since this loan is from a lifeco and priced over the Treasury rate instead of LIBOR or the swap rate, we will change that to a 30 / 360 day count convention. The 30 / 360 calculation method used in _cred_ is the method Excel uses.

In [6]:
from cred import thirty360

loan.year_frac = thirty360

Finally, since insurance companies are so risk averse, we'll add amortization on a 30-year schedule to meet their concerns of takeout risk.

In [7]:
loan.amort_periods = 360
loan.schedule()

,start_date,end_date,payment_date,bop_principal,interest_rate,interest_payment,principal_payment,payment,eop_principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,1.000000e+08,0.0219,97333.333333,0.000000e+00,9.733333e+04,1.000000e+08
1,2020-05-31,2020-06-30,2020-06-30,1.000000e+08,0.0219,182500.000000,1.966930e+05,3.791930e+05,9.980331e+07
2,2020-06-30,2020-07-31,2020-07-31,9.980331e+07,0.0219,182141.035307,1.970519e+05,3.791930e+05,9.960626e+07
3,2020-07-31,2020-08-31,2020-08-31,9.960626e+07,0.0219,181781.415504,1.974116e+05,3.791930e+05,9.940884e+07
4,2020-08-31,2020-09-30,2020-09-30,9.940884e+07,0.0219,181421.139394,1.977718e+05,3.791930e+05,9.921107e+07
5,2020-09-30,2020-10-31,2020-10-30,9.921107e+07,0.0219,181060.205781,1.981328e+05,3.791930e+05,9.901294e+07
6,2020-10-31,2020-11-30,2020-11-30,9.901294e+07,0.0219,180698.613464,1.984944e+05,3.791930e+05,9.881444e+07
7,2020-11-30,2020-12-31,2020-12-31,9.881444e+07,0.0219,180336.361241,1.988566e+05,3.791930e+05,9.861559e+07
8,2020-12-31,2021-01-31,2021-01-29,9.861559e+07,0.0219,179973.447907,1.992195e+05,3.791930e+05,9.841637e+07


## Putting it all together

Congratulations, you've created a customized loan schedule using _cred_ !  You can export the loan schedule to an Excel file in your working directory by running `loan.schedule().to_excel('my_file_name.xlsx')`. The last code block below compiles everything we've done into one initialization statement and formats the output to look nicer.

Visit the documentation at [cred.readthedocs.io](https://cred.readthedocs.io/en/latest/) to learn more about how you can extend loan objects to build dynamic floating rate schedules, calculate covenant tests, and evaluate other loan metrics using subclasses.

In [8]:
from cred import FixedRateBorrowing, Monthly, thirty360, FederalReserveHolidays, modified_following

loan = FixedRateBorrowing(start_date=date.today() + relativedelta(months=1, day=15),
                          first_reg_start=date.today() + relativedelta(months=1, day=31),
                          end_date=date.today() + relativedelta(months=25, day=15),
                          freq=Monthly(),
                          initial_principal=100_000_000,
                          coupon=yc.yield_for_delta(relativedelta(months=12)) + 0.02,
                          amort_periods=360,
                          pmt_convention=modified_following,
                          year_frac=thirty360,
                          holidays=FederalReserveHolidays())

# formatting
schedule = loan.schedule()
schedule.columns = ['Start Date', 'End Date', 'Payment Date', 'Start Principal', 'Interest Rate', 'Interest Payment', 'Principal Payment', 'Total Payment', 'End Principal']
schedule = schedule.style.format({'Start Principal': '${:,.0f}', 'Interest Rate': '{:.2%}', 'Interest Payment': '${:,.0f}', 'Principal Payment': '${:,.0f}', 'Total Payment': '${:,.0f}', 'End Principal': '${:,.0f}'})

schedule

,Start Date,End Date,Payment Date,Start Principal,Interest Rate,Interest Payment,Principal Payment,Total Payment,End Principal
index,,,,,,,,,
0,2020-05-15,2020-05-31,2020-05-15,"$100,000,000",2.19%,"$97,333",$0,"$97,333","$100,000,000"
1,2020-05-31,2020-06-30,2020-06-30,"$100,000,000",2.19%,"$182,500","$196,693","$379,193","$99,803,307"
2,2020-06-30,2020-07-31,2020-07-31,"$99,803,307",2.19%,"$182,141","$197,052","$379,193","$99,606,255"
3,2020-07-31,2020-08-31,2020-08-31,"$99,606,255",2.19%,"$181,781","$197,412","$379,193","$99,408,844"
4,2020-08-31,2020-09-30,2020-09-30,"$99,408,844",2.19%,"$181,421","$197,772","$379,193","$99,211,072"
5,2020-09-30,2020-10-31,2020-10-30,"$99,211,072",2.19%,"$181,060","$198,133","$379,193","$99,012,939"
6,2020-10-31,2020-11-30,2020-11-30,"$99,012,939",2.19%,"$180,699","$198,494","$379,193","$98,814,445"
7,2020-11-30,2020-12-31,2020-12-31,"$98,814,445",2.19%,"$180,336","$198,857","$379,193","$98,615,588"
8,2020-12-31,2021-01-31,2021-01-29,"$98,615,588",2.19%,"$179,973","$199,220","$379,193","$98,416,368"


***

<a id='pythonenv'><a/>
## Setting up the Python environment

This assumes Python 3 is already installed. If not, install from [python.org](https://www.python.org). To check if it's already installed, run `which python` in the terminal.

Open up Terminal (mac) or Command Prompt (Windows) and navigate to the location where you want to set up your project. Type:

``` 
mkdir cred_guide
cd cred_guide 
```

to create a new folder called "cred_guide" and then move inside the new folder. This is where we will create our virtual environment and store our project. 

Next, we'll set up our virtual environment. Virtual environments are isolated environments where you can install and run libraries for a specific project without worrying about messing up packages or dependencies for other projects. It isn't required, but it's best practice. Real Python has a great [primer on virtual environments](https://realpython.com/python-virtual-environments-a-primer/). We'll walk through Virtualenv here since it comes with the base version of Python 3, but there are several ways to set up virtual environments. [Conda](https://docs.conda.io/en/latest/) is another popular option.

Create and activatethe virtual environment by running the following in the termnial.

```
python -m venv venv

// activate on mac
source venv/bin/activate

// activate on windows
C:\path\to\cred_guide\venv\Scripts\activate.bat
```

The terminal prompt should change to something like `(base) :UserName$` to show that the virtual environment has been activated.

Finally, install the python packages necessary to run this tutorial:

```
pip install python-dateutil
pip install git+https://github.com/jordanhitchcock/treasurydotgovyc
pip install git+https://github.com/jordanhitchcock/cred

python  # Starts Python. From here on out, everything is Python code.

from datetime import date
from dateutil.relativedelta import relativedelta
 
from treasurydotgovyc import YieldCurve
from cred import FixedRateBorrowing, Monthly
```

Great, you're ready to jump back in at the [Creating the loan schedule](#loanschedule) section!